MNIST

DataLoader, Transformation

Multilayer Neural Net, Activation function

Loss and Optimizer

Training loop (batch training)

Model evaluation

GPU support

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import sys
import torch.nn.functional as F
# from torch.utils.tensorboard import SummaryWriter
# writer = SummaryWriter('./runs/mnist2')

In [3]:
# device config
# device = torch_directml.device()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# hyper parameters
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 2
batch_size = 64   
learning_rate = 0.01

# Transform data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])

# MNIST
train_dataset = torchvision.datasets.MNIST(root="./data", train=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

examples = iter(train_loader)
samples, labels = next(examples)
print(samples.shape, labels.shape)

# for i in range(6):
#     plt.subplot(2, 3, i + 1)
#     plt.imshow(samples[i][0], cmap='gray')
# plt.show()
img_grid = torchvision.utils.make_grid(samples)
# writer.add_image('mnist_images', img_grid)
# writer.close()
# sys.exit()



class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out
    
model = NeuralNet(input_size, hidden_size, num_classes)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# writer.add_graph(model, samples.reshape(-1, 28*28))
# writer.close()
# sys.exit()


# training loop
n_total_steps = len(train_loader)

running_loss = 0.0
running_correct = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # 100, 1, 28, 28 -> 784
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        # forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        running_correct += (predicted == labels).sum().item()

        if (i+1) % 100 == 0:
            
            print(f'Epoch: [{epoch+1} / {num_epochs}], Step: [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

            # writer.add_scalar('training loss', running_loss / 100, epoch * n_total_steps + i)
            # writer.add_scalar('accuracy', running_correct / 100, epoch * n_total_steps + i)

            running_loss = 0.0
            running_correct = 0



# test
preds = []
pred_labels = []
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predictions = torch.max(outputs, 1)
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()

        class_predictions = [F.softmax(output, dim=0) for output in outputs]

        preds.append(class_predictions)
        pred_labels.append(predictions)

    preds = torch.cat([torch.stack(batch) for batch in preds])
    labels = torch.cat(pred_labels)

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy = {acc}')

    classes = range(10)
    for i in classes:
        labels_i = labels == i
        preds_i = preds[:, i]
        # writer.add_pr_curve(str(i), labels_i, preds_i, global_step=0)
        # writer.close()

# torch.save(model.state_dict(), "mnist_fnn.pth")

# Accuracy = 94.61

torch.Size([64, 1, 28, 28]) torch.Size([64])
Epoch: [1 / 2], Step: [100/938], Loss: 0.2348
Epoch: [1 / 2], Step: [200/938], Loss: 0.2884
Epoch: [1 / 2], Step: [300/938], Loss: 0.2427
Epoch: [1 / 2], Step: [400/938], Loss: 0.1160
Epoch: [1 / 2], Step: [500/938], Loss: 0.1046
Epoch: [1 / 2], Step: [600/938], Loss: 0.4386
Epoch: [1 / 2], Step: [700/938], Loss: 0.2848
Epoch: [1 / 2], Step: [800/938], Loss: 0.2228
Epoch: [1 / 2], Step: [900/938], Loss: 0.1589
Epoch: [2 / 2], Step: [100/938], Loss: 0.0357
Epoch: [2 / 2], Step: [200/938], Loss: 0.0406
Epoch: [2 / 2], Step: [300/938], Loss: 0.2429
Epoch: [2 / 2], Step: [400/938], Loss: 0.4032
Epoch: [2 / 2], Step: [500/938], Loss: 0.6934
Epoch: [2 / 2], Step: [600/938], Loss: 0.1057
Epoch: [2 / 2], Step: [700/938], Loss: 0.5872
Epoch: [2 / 2], Step: [800/938], Loss: 0.1866
Epoch: [2 / 2], Step: [900/938], Loss: 0.2459
Accuracy = 94.61
